In [3]:
import os
from datetime import datetime
import plotly.offline as offline
import app
from app.bot import macd
app.set_db(["localhost", "45.79.176.125"][0])
db = app.get_db()

pair='WANBTC'
units = ('minutes', 'm', 'T')
n_units, n_periods = 5, 400

fig = macd.plot(pair, units, n_units, n_periods)
file = "{}_{}_{}.html".format(pair, str(n_units)+units[1], str(datetime.now().date()))
offline.plot(fig, filename=file)
os.system("mv {} charts/".format(file))

0